# Benchmark experiment on Behravan Algorithm

This notebook collects all the code needed to run the benchmark experiment on the algorithm named SVM-Behravan in the paper [1].

The paper can be accessed from here: https://doi.org/10.1038/s41598-018-31573-5

The original source code can be accessed here: https://github.com/hambeh/breast-cancer-risk-prediction

For this implementation we kept the original source code but we updated it in Python 3.
Details on the hyperparameters' choices implemented in the following can be found in subection Benchmark Algorithms within Materials and Methods section.


### References
[1] Behravan, H., Hartikainen, J.M., Tengström, M. et al. Machine learning identifies interacting genetic variants contributing to breast cancer risk: A case study in Finnish cases and controls. Sci Rep 8, 13149 (2018). 

In [1]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import numpy as np
import pickle
from sklearn.metrics import auc
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone
from sklearn.metrics import precision_recall_curve
from sklearn import svm
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import average_precision_score, roc_auc_score
import time
import pandas as pd


from hiprs import* # Library for high-order interaction aware Polygenic Risk Scores
import snps # Auxiliary library for the simulation of genomic data


import warnings
warnings.filterwarnings("ignore")

## Functions

 We collected in the following all functions needed to run the experiment as available in the Github repository of the authors of the algorithm to facilitate inspection.

In [2]:
# Stage 2 functions
def cal_XGboost(X_train, Y_train, model, x_test, y_test):
    # fitting an XGBoost model and returning feature importance (gain)
    model_XGboost = clone(model)
    eval_set = [(x_test, y_test)]
    model_XGboost.fit(X_train, Y_train, verbose = False, early_stopping_rounds=(model_XGboost.n_estimators)/10, eval_metric="auc", eval_set=eval_set)
    #print('XGboost done')
    return model_XGboost.get_booster().get_score(importance_type='gain')

def all_results_SVM(XX_train,YY_train,XX_validation,YY_validation,indices):
#    classifier = clone(model)
#    print(model)
    classifier = svm.SVC(probability=True, random_state=3, kernel='linear', C=1.5, class_weight='balanced')
    classifier.fit(XX_train[:,indices], YY_train)
    ts_score = classifier.predict_proba(XX_validation[:,indices])
    return ts_score[:,1]


def cal_XGboost_feature_importance(X_train,Y_train, indices, model, X_test, Y_test):
    # # initial sort: algorithm 1 step 1
    model_1 = clone(model)
    eval_set = [(X_test[:,indices], Y_test)]
    model_1.fit(X_train[:,indices], Y_train,  verbose = False, early_stopping_rounds=(model_1.n_estimators)/10, eval_metric="auc", eval_set=eval_set)
    # now sort based on gain
    scores_key_values = model_1.get_booster().get_score(importance_type='gain')
    index_non_zero = list()
    for i in range(len(scores_key_values.keys())): # getting indices of used features in xgboost in [0, len(indices)]
        index_non_zero.append(np.int64(list(scores_key_values.keys())[i][1:]))# indices of keys
    sorted_values = np.argsort(list(scores_key_values.values()))[::-1] # argsorting based on gain and getting corresponding top indices.
    from_top_temp = indices[np.array(index_non_zero)[sorted_values]]   # in range [0,125041]
    zir_from_top = np.array(list(set(indices)^set(indices[np.array(index_non_zero)[sorted_values]])))
    from_top = np.concatenate((from_top_temp, zir_from_top), axis=0)
    return from_top

# implementation of algorithm 1
def second_cal_XGboost_feature_importance4(XX_train,YY_train, SNPs_indices_sorted_main, M, K, N, model, X_test, Y_test):
    SNPs_indices_sorted= SNPs_indices_sorted_main.copy()
    model1 = clone(model)
    model2 = clone(model)
    temp = []

    for i in K:

        if(M+i <= len(SNPs_indices_sorted)//2):
            eval_set1 = [(X_test[:,SNPs_indices_sorted[:M+i]], Y_test)]
            model1.fit(XX_train[:,SNPs_indices_sorted[:M+i]], YY_train, verbose = False, early_stopping_rounds=(model1.n_estimators)/10, eval_metric="auc",
            eval_set=eval_set1) # from top

            eval_set2 = [(X_test[:,SNPs_indices_sorted[-M-i:]], Y_test)]
            model2.fit(XX_train[:,SNPs_indices_sorted[-M-i:]], YY_train, verbose = False, early_stopping_rounds=(model2.n_estimators)/10, eval_metric="auc",
            eval_set=eval_set2)# from bottom

            scores4_key_values = model1.get_booster().get_score(importance_type='gain')
            index_non_zero4 = list()
            for uu in range(len(scores4_key_values.keys())): # getting indices of used features in xgboost in [0, len(indices)]
                index_non_zero4.append(np.int64(list(scores4_key_values.keys())[uu][1:]))# indices of keys
            sorted_values4 = np.argsort(list(scores4_key_values.values()))[::-1] # argsorting based on gain and getting corresponding top indices.
            M_top = []
            M_top_temp = SNPs_indices_sorted[:M+i][np.array(index_non_zero4)[sorted_values4]]
            zir_top = np.array(list(set(SNPs_indices_sorted[:M+i])^set(SNPs_indices_sorted[:M+i][np.array(index_non_zero4)[sorted_values4]])))
            M_top = np.concatenate((M_top_temp, zir_top), axis=0)

            scores5_key_values = model2.get_booster().get_score(importance_type='gain')
            index_non_zero5 = list()
            for uu in range(len(scores5_key_values.keys())): # getting indices of used features in xgboost in [0, len(indices)]
                index_non_zero5.append(np.int64(list(scores5_key_values.keys())[uu][1:]))# indices of keys
            sorted_values5 = np.argsort(list(scores5_key_values.values()))[::-1] # argsorting based on gain and getting corresponding top indices.
            M_bottom_temp = SNPs_indices_sorted[-M-i:][np.array(index_non_zero5)[sorted_values5]]
            zir_bottom = np.array(list(set(SNPs_indices_sorted[-M-i:])^set(SNPs_indices_sorted[-M-i:][np.array(index_non_zero5)[sorted_values5]])))
            M_bottom = np.concatenate((M_bottom_temp, zir_bottom), axis=0)

            # replace m top with m bottom  rankked snps
            SNPs_indices_sorted[:M+i] = M_top # resorting based on new M-top
            SNPs_indices_sorted[len(SNPs_indices_sorted)-M-i:len(SNPs_indices_sorted)] = M_bottom # resoritng based on new M-bottom

            SNPs_indices_sorted[M+i-N:M+i] = M_bottom[0:N] # exchange N best M_bottom with N worst M_top 
            SNPs_indices_sorted[len(SNPs_indices_sorted)-M-i:len(SNPs_indices_sorted)-M-i+N] = M_top[M+i-N:M+i] # exchange N worst M_top with N best M_bottom 
            temp = SNPs_indices_sorted

    return temp

def Tune_stage2(xgboost_scores, X_train, Y_train, X_test, Y_test, model): # From test NOT CV

    model_Tune_stage2 = clone(model)
    average_index_non_zero = list()
    for i in range(len(xgboost_scores.keys())): #getting indices of selected features from training set. Indices are in [0,125041]
        average_index_non_zero.append(np.int64(list(xgboost_scores.keys())[i][1:]))

    MM = [2] # window size (Algorithm 1 step 2)
    K_increament = [1] # adaptively increase window size (Algorithm 1 step 5)
    NN = [1] # let's fix N. algorithm 1 step 4 
    global_returned_sorted = list()
    tot_roc = list()
    # let's fix m, n and k
    SNPs_indices_sorted_main = cal_XGboost_feature_importance(X_train, Y_train, np.array(average_index_non_zero), model_Tune_stage2, X_test, Y_test)# initial sorting
    SNPs_indices_sorted_main = np.int64(SNPs_indices_sorted_main)
    for M in MM:
        for KK in K_increament:
            for N in NN:
                # initial N=sort
                K = []
                K = list(map(lambda x: x*KK , list(range(len(SNPs_indices_sorted_main)//2))))
                returned_sorted4 = second_cal_XGboost_feature_importance4(X_train, Y_train, SNPs_indices_sorted_main, M, K, N, model_Tune_stage2, X_test, Y_test)
                if(len(returned_sorted4)):
                    selected = np.int64(((np.array(list(range(100)))+1)/float(100))*len(returned_sorted4)) # 1% 2%
                    selected = np.unique(selected)
                    for ii in selected:
                        if(ii!=0):
#                            print(ii)
                            ts_score1=all_results_SVM(X_train,Y_train, X_test, Y_test, returned_sorted4[:ii])
                            # specific M, K, N, ii and CV
                            #print('M: ' + str(M)+ ' K: ' + str(KK) + ' N: ' + str(N) + ' ii: ' +  str(ii))
                            global_returned_sorted.append(returned_sorted4[:ii])
                            precision2, recall2, _ = precision_recall_curve(Y_test, ts_score1)
                            tot_roc.append(auc(recall2, precision2))
#                            print(auc(recall2, precision2))
                            

    best_indices_auc_recall = global_returned_sorted[np.argsort(tot_roc)[::-1][0]]
    return best_indices_auc_recall

def build_XGboost():
    model_x = XGBClassifier(nthread=1,seed=0,n_estimators=100,max_depth=2,learning_rate=0.01,subsample=0.4)
    return model_x


## Algorithm 

Running this cell performs all the 30 experiments on simulated data to produce the results reported in the paper. At each loop a new simulated dataset is generated via the custom `generate()` function, available in the `hiprs` packege, using one of the 30 seeds set for the experiments on hiPRS algorithm as well. Generated datasets are saved in the directory of the present notebook where they can be accessed by the R code running the other benchmark experiment (`glinternet.R`).  

In [3]:
seeds = list(range(0,30))

avg_prec_training_final = list()
avg_prec_dev_final = list()
avg_prec_test_final = list()
avg_auc_test_final = list()
tot_runtime = list()
selected_snps = list()


for seed in seeds:

    tot_average_precisionTR = list()
    tot_average_precisionDev = list()
    tot_average_precisionTS = list()
    best_indices_cvt_auc_recall = list()
    tot_AUC_TS = list()

    Data = generate(1500, 15, 0.01, seed=seed)
    
    Data.to_csv("hiprs_simdata_%d.csv" % seed)

    x = Data.iloc[:1000,:-1].values
    y = Data.iloc[:1000,-1].values
    x_cv = Data.iloc[1000:,:-1].values
    y_cv = Data.iloc[1000:,-1].values

    print('Executing seed = %d' %(seed))

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

    # Fix model for all 5 cvs
    # defining an XGBoost model from obtained optimal hyperparameters
    model = XGBClassifier(nthread=1,seed=0,n_estimators=500,max_depth=6,learning_rate=0.01,subsample=0.3)

    best_indices_cv_auc_recall = list()
    # Important: same train and test split as xgboost optimization codes  by fixing random seed
    
    np.random.seed(seed)
    
    start_time = time.time()
    
    for train, test in cv.split(x, y):  
        X_train = x[train]
        Y_train = y[train]
        X_test = x[test]
        Y_test = y[test]
        xgboost_scores1 = cal_XGboost(X_train, Y_train, model, X_test, Y_test)
        best_indices_au_recall = Tune_stage2(xgboost_scores1,X_train,Y_train,X_test,Y_test,model)
        best_indices_cv_auc_recall.append(best_indices_au_recall)

    best_indices_cvt_auc_recall.append(best_indices_cv_auc_recall)

    best_indices = best_indices_cvt_auc_recall

    indices_new = []
    temp = list()
    for j in range(len(best_indices)):
        for k in range(len(best_indices[j])):
    #        temp.append(len(best_indices[j][k]))
            indices_new.append(list(best_indices[j][k]))

    indices_new1 = np.unique(np.concatenate(indices_new))
    
    selected_snps.append(indices_new)

    counter = -1

    np.random.seed(seed)
    
    for train, test in cv.split(x, y):  # accessing train and test for stage 2 parameter tuning 
        X_train = x[train]
        Y_train = y[train]
        X_test = x[test]
        Y_test = y[test]
        counter = counter+1
        if(len(indices_new[counter])):

             # training
            ts_scoreL1=all_results_SVM(X_train, Y_train, X_train, Y_train, indices_new[counter])           
            tot_average_precisionTR.append(average_precision_score(Y_train, ts_scoreL1))             
             # val
            ts_scoreL1=all_results_SVM(X_train, Y_train, X_test, Y_test, indices_new[counter])           
            tot_average_precisionDev.append(average_precision_score(Y_test, ts_scoreL1))               
             # test             
            ts_scoreL1=all_results_SVM(X_train, Y_train, x_cv, y_cv, indices_new[counter])           
            tot_average_precisionTS.append(average_precision_score(y_cv, ts_scoreL1))
            tot_AUC_TS.append(roc_auc_score(y_cv, ts_scoreL1))


    end_time = time.time()
    
    runtime = round(end_time - start_time, 3)
    
    tot_runtime.append(runtime)
    
    avg_prec_training_in = np.mean(tot_average_precisionTR)
    avg_prec_dev_in = np.mean(tot_average_precisionDev)
    avg_prec_test_in = np.mean(tot_average_precisionTS)
    avg_auc_test_in = np.mean(tot_AUC_TS)
    
    avg_prec_training_final.append(avg_prec_training_in)
    avg_prec_dev_final.append(avg_prec_dev_in)
    avg_prec_test_final.append(avg_prec_test_in)
    avg_auc_test_final.append(avg_auc_test_in)



    
ds = pd.DataFrame([avg_prec_test_final, avg_auc_test_final, tot_runtime]).T
ds.columns = ['avg_test_precision', 'avg_test_AUC', 'runtime_sec']
ds.to_csv('Behravan_results_NEW.csv')

pd.DataFrame(selected_snps).to_csv('selected_snps_behravan_NEW.csv')

    
print(str('Train Average precision: ')  + str(np.mean(avg_prec_training_final)*100)+ str('std: ') + str(np.std(avg_prec_training_final)))
print(str('Val Average precision: ')  + str(np.mean(avg_prec_dev_final)*100)+ str('std: ') + str(np.std(avg_prec_dev_final)))
print(str('Test Average precision: ')  + str(np.mean(avg_prec_test_final)*100)+ str('std: ') + str(np.std(avg_prec_test_final)))
print('Average Execution runtime for each dataset: %.3f seconds' % np.mean(tot_runtime))

Executing seed = 0
Executing seed = 1
Executing seed = 2
Executing seed = 3
Executing seed = 4
Executing seed = 5
Executing seed = 6
Executing seed = 7
Executing seed = 8
Executing seed = 9
Executing seed = 10
Executing seed = 11
Executing seed = 12
Executing seed = 13
Executing seed = 14
Executing seed = 15
Executing seed = 16
Executing seed = 17
Executing seed = 18
Executing seed = 19
Executing seed = 20
Executing seed = 21
Executing seed = 22
Executing seed = 23
Executing seed = 24
Executing seed = 25
Executing seed = 26
Executing seed = 27
Executing seed = 28
Executing seed = 29
Train Average precision: 22.263664235408662std: 0.047861906406610616
Val Average precision: 23.434269786963867std: 0.053491525596715155
Test Average precision: 20.99510344729047std: 0.04236904586802405
Average Execution runtime for each dataset: 10.086 seconds
